<a href="https://colab.research.google.com/github/ryanoleary26/VirusTotal-Payload-Scanner/blob/master/VirusTotal_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing Honeypot Payloads with the VirusTotal API

Any files that are given to this script in a .ZIP file will be indivually sent to VirusTotal for scanning using [their v2 API](https://developers.virustotal.com/reference).

#Features



1. Displays scan reports from over [70 different Antivirus products](https://support.virustotal.com/hc/en-us/articles/115002146809-Contributors).
2. Provides a URL to each indivudual scan performed.
3. Provides statitistics on total scans performed and total positive results.
4. Ideal for analysis of malicious files collected through honeypots.
5. Basic API quota usage detection
6. Option to show scan results from specific Antivirus scanners instead of an overall summary.

#Prerequesites

1. Python
2. [Requests](https://requests.readthedocs.io/en/master/) module

#How to use

1. To use this script for yourself you will need to obtain a free public API key by [creating an account on the VirusTotal website](https://www.virustotal.com/gui/join-us).
2. Enter your API key on line 8, see below:

  ```
  #Enter API key here ########
  api_key = '<API KEY HERE>' #
  ############################
  ```
3. Enter the file path of the ZIP file you wish to use on line 17, see below. Note that currentlyy files that are contained within sub-directories of the ZIP file are currently not scanned. To get round this seperate groups of files into seperate ZIP files until I implement this feature.
```
  #Enter the location of your .ZIP file here ######
  with ZipFile('<FILE PATH HERE>', 'r') as zipObj:#
  #################################################
```
4. Line 12 contains a dictionary of Antivirus (AV) names to include in the scan output. Add or remove Antivirus names from this dictionary as you please. The default configuration will include Avast, AVG, BitDefender, FireEye, F-Secure, McAfee and Microsoft in the specific scan result output. This **IS** case sensitive.
```
#Customise the scan output by changing the included AV names here. This IS case and space sensitive.
antiviruses = {'Avast', 'AVG', 'BitDefender', 'FireEye', 'F-Secure', 'Malwarebytes', 'McAfee', 'Microsoft'}
```
5. Run the script and enjoy the sexy Antivirus results - Happy Virus Hunting!








In [3]:
import json
from zipfile import ZipFile
import time
import requests
import os

#Enter API key here ########
api_key = '<API KEY HERE>' #
############################ 

#Customise the scan output by changing the included AV names here. This IS case and space sensitive.########
antiviruses = {'Avast', 'AVG', 'BitDefender', 'FireEye', 'F-Secure', 'Malwarebytes', 'McAfee', 'Microsoft'}#
############################################################################################################

#Enter the location of your .ZIP file here ######
with ZipFile('<FILE PATH HERE>', 'r') as zipObj:#
#################################################

  # Extract all the contents of zip file in different directory
  hashList = zipObj.namelist()

print('##############################################################################################################\n')
payloadNo = 0

#Tracks overall stats
class Stats:
  def __init__ (self):
    self.positiveResult = 0
    self.negativeResult = 0
    self.noResult = 0
    self.extensionsFound = {}

  positiveResult = 0
  negativeResult = 0
  noResult = 0
  extensionsFound = {}

try:
  for payload in hashList:
      payloadNo += 1
      #Makes request to virustotal to scan file
      url = 'https://www.virustotal.com/vtapi/v2/file/report'
      params = {'apikey': api_key, 'resource': payload, 'allinfo': 1}
      try:
        response = requests.get(url, params=params)
        vt_response = response.json()
      except:
        if response.status_code == 204:
        #Handles the 204 response code
          print('Recieved HTTP Response Code:', response.status_code, '-- Indicates that API usage quota has been used up.\nRefer to the VirusTotal API documentation for error description.\nDocumentation link: https://developers.virustotal.com/reference#api-responses')
          break
        else:
          print('JSON Decoder Error')
          break

      #Parsing the JSON response
      print('Scanning Payload {0}/{1}: {2}'.format( payloadNo, len(hashList),payload))
      
      if vt_response['response_code'] == 1:
        print('Scan initiated at', vt_response['scan_date'])
        #Doesn't post AV results if there are zero positive results
        if vt_response['positives'] == 0:
          print('\nAll scans returned negative.')
          Stats.negativeResult += 1
        else:
          print('=== Antivirus Results ===')
          #Loops through selected AVs for detection results
          for av in antiviruses:
            try:
              print(av, '-> Detected:', vt_response['scans'][av]['detected'], ' Result:', vt_response['scans'][av]['result'])
            except KeyError:
              #AVs included in 'antiviruses' might not always return scan results, which will cause a TypeError
              print(av, 'could not return any test results.')

          #Icrease the threshold of a 'positive' match for malware to something like 5? 
          #to avoid a single rouge AV from spoiling results    
          Stats.positiveResult += 1

        print('\n=== Scan Results ===')
        print('Total Scans:', vt_response['total'])
        print('Total Positive Results:', vt_response['positives'])
        print('First seen:', vt_response['first_seen'])
        print('Times Submitted:', vt_response['times_submitted'])
        print('Scan Permalink: ', vt_response['permalink'], '\n')
      
        print('=== EXIF Data provided by https://exiftool.org/ ===')
        try:
          print('File Type/Extension: {} (.{})'.format(vt_response['additional_info']['exiftool']['FileType'], vt_response['additional_info']['exiftool']['FileTypeExtension']))
          print('File Size:', round(vt_response['size']/1024/1024,2), 'MB')
          print('Description:', vt_response['additional_info']['magic'])
          print('Target Operating System:', vt_response['additional_info']['exiftool']['OperatingSystem'])
          
          ######
        
          # Stats.extensionsFound.setdefault('ext', {})[vt_response['additional_info']['exiftool']['FileType']] = 1

          count = Stats.extensionsFound.get(vt_response['type'], 0)
          Stats.extensionsFound[vt_response['type']] = count + 1
          
          ######

        except KeyError as err:
          print('\nSome EXIF data was not found :(\n')
        
      #Handles other response codes: https://developers.virustotal.com/reference#api-responses
      elif vt_response['response_code'] != 1:
        print('===========================================================================\n',vt_response['verbose_msg'],'\n===========================================================================')
        Stats.noResult += 1
      print('\n##############################################################################################################\n')

      #Sleeps script to comply with API user agreement
      #Set to 15 for public API, 4.5 for Academic/Premium API
      time.sleep(4.5)

  #Generic exception handling :/
except KeyboardInterrupt:
  print('Keyboard Interrupt')
print('Scan complete')
#Show intersting scan stats
print('=== Scan Report ===')
print('{0} files were scanned'.format(payloadNo))
print('{0} positive matches were found for malicous files'.format(Stats.positiveResult))
print('{0} negative matches were found for malicous files'.format(Stats.negativeResult))
print('{0} scans had no result due to files still being analysed.'.format(Stats.noResult))
print('List of different file extensions found:')

#Show each file extension found
for ext in Stats.extensionsFound:
  print('   {0}: {1}'.format(ext,Stats.extensionsFound[ext]))

##############################################################################################################

Scanning Payload 1/1391: 0013d4e2ee63e2be651c99071bb3207c5acef4b9aa1e135e8f19eba656e68d1a
Scan initiated at 2020-05-01 16:48:52

All scans returned negative.

=== Scan Results ===
Total Scans: 58
Total Positive Results: 0
First seen: 2020-01-13 19:08:22
Times Submitted: 3
Scan Permalink:  https://www.virustotal.com/file/0013d4e2ee63e2be651c99071bb3207c5acef4b9aa1e135e8f19eba656e68d1a/analysis/1588351732/ 

=== EXIF Data provided by https://exiftool.org/ ===
File Type/Extension: GZIP (.gz)
File Size: 2.77 MB
Description: gzip compressed data, from Unix, last modified: Mon Jan 06 15:07:35 2020
Target Operating System: Unix

##############################################################################################################

Scanning Payload 2/1391: 00ce2b7587eb0ff6663f57ed5d59ceabacad865f1c06b4ca8c8c7416b74b7b5a
Scan initiated at 2019-09-27 13:46:19

All scans returne